In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import re
import ipywidgets as widgets
from IPython.display import display

In [ ]:
home_path = '/content/drive/MyDrive/KMITL/Project/Data'

file_dropdown = widgets.Dropdown(
    options=[file_name for file_name in os.listdir(home_path) if file_name.endswith('.csv')],
    description='select dataset',
)
display(file_dropdown)

Dropdown(description='select dataset', options=('full dataset.csv',), value='full dataset.csv')

In [ ]:
print(f'selected file: {file_dropdown.value}')
file_path = os.path.join(home_path, file_dropdown.value)
df = pd.read_csv(file_path)

df = df[['News','Symbol','Keyword (Aspect)','Polarity (Opinion)','Sentiment']]
df.head()

selected file: full dataset.csv


,News,Symbol,Keyword (Aspect),Polarity (Opinion),Sentiment
0,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,HMPRO,E-refune,หนุน,Positive
1,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,HMPRO,หุ้น,รับผลดี,Positive
2,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,HMPRO,Q1,โต,Positive
3,ILM กรุงศรี พัฒนสิน มองอาจมี upside หลัง SSSG ...,ILM,SSSG,เติบโตดี,Positive
4,IHLเป๋าตุงมกราทันที รับรู้รองเท้าแบรนด์ดัง,IHL,มกรา,เป๋าตุง,Positive


In [ ]:
df[df.isnull().any(axis=1)]

,News,Symbol,Keyword (Aspect),Polarity (Opinion),Sentiment
30,วิพากษ์KEXยังขาดทุน รู้ทันปรับผู้ถือหุ้นใหญ่,KEX,NaN,ขาดทุน,Negative
230,AUCT ชี้รถมือสองโต ลุยประมูลออนไลน์,AUCT,NaN,โต,Positive
343,WHA ยังแข็งแกร่ง โบรกให้เป้าหมาย 5.70 บ.,WHA,NaN,ยังแข็งแกร่ง,Positive
421,ตรุษจีน3.4หมื่นล้าน GFPTลั่นคึกคัก,GFPT,NaN,คึกคัก,Positive
435,SPRC โบรกมองดีขึ้น Q1/67 ให้เป้า 8.60-11.75 บ.,SPRC,NaN,ดีขึ้น,Positive
...,...,...,...,...,...
8273,CPF ไตรมาส 3/66 ขาดทุน 1.8 พันล้าน เหตุต้นทุนส...,CPF,NaN,ขาดทุน,Negative
8286,JKN ดิ่งฟลอร์ 29% หลังยื่นขอฟื้นฟูกิจการ ‘โบรก...,JKN,NaN,ดิ่ง,Negative
8288,JKN ไม่รอด! 'ยื่นล้มละลาย' ขอฟื้นฟูกิจการ แม้ส...,JKN,NaN,ไม่รอด,Negative
8303,TRUE ร่วงกว่า 6% กังวลขายหุ้นกู้ไม่ได้ตามเป้า ...,TRUE,NaN,ร่วง,Negative


In [ ]:
sent_tag = {
    'positive':'<POS>',
    'negative':'<NEG>',
    'neutral': '<NEU>',
}
SENTIMENT_TAGS = [v for k,v in sent_tag.items()] # ['<POS>','<NEG>','<NEU>']
SPECIAL_TAGS = ['<SYMBOL>','<ASPECT>','<OPINION>'] + SENTIMENT_TAGS

In [ ]:
dataset = pd.DataFrame(columns=['text','label'])
dataset['text'] = df['News']

In [ ]:
def to_label_text(label):
  label_text = ''
  if label.Symbol:
    label_text += f'<SYMBOL> {label.Symbol} '
  if label.Aspect:
    label_text += f'<ASPECT> {label.Aspect} '
  if label.Opinion:
    label_text += f'<OPINION> {label.Opinion} '
  if label.Sentiment:
    label_text += sent_tag[label.Sentiment.lower()]

  return label_text

In [ ]:
print(df.columns)

Index(['News', 'Symbol', 'Keyword (Aspect)', 'Polarity (Opinion)',
       'Sentiment'],
      dtype='object')


In [ ]:
def to_label_text(row): # Changed 'label' to 'row' for clarity
  label_text = ''
  if row['Symbol']: # Accessing values using column names
    label_text += f'<SYMBOL> {row["Symbol"]} '
  if row['Keyword (Aspect)']:
    label_text += f'<ASPECT> {row["Keyword (Aspect)"]} '
  if row['Polarity (Opinion)']:
    label_text += f'<OPINION> {row["Polarity (Opinion)"]} '
  if row['Sentiment']:
    try:  # Attempt to convert to lowercase
      label_text += sent_tag[str(row['Sentiment']).lower()] # Changed to str(row['Sentiment']) to ensure it's a string
    except (KeyError, AttributeError):  # Handle errors if lowercase conversion or key lookup fails
      label_text += ''  # Or any other desired behavior for invalid values
      print(f"Warning: Invalid Sentiment value: {row['Sentiment']}")  # Optional: Print a warning message

  return label_text

dataset['label'] = df.apply(lambda x: to_label_text(x), axis=1) # Applying the updated function

In [ ]:
dataset

,text,label
0,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,<SYMBOL> HMPRO <ASPECT> E-refune <OPINION> หนุ...
1,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,<SYMBOL> HMPRO <ASPECT> หุ้น <OPINION> รับผลดี...
2,HMPRO เคจีไอฯ ชี้เป็นหนึ่งในหุ้นรับผลดีมาตรการ...,<SYMBOL> HMPRO <ASPECT> Q1 <OPINION> โต <POS>
3,ILM กรุงศรี พัฒนสิน มองอาจมี upside หลัง SSSG ...,<SYMBOL> ILM <ASPECT> SSSG <OPINION> เติบโตดี ...
4,IHLเป๋าตุงมกราทันที รับรู้รองเท้าแบรนด์ดัง,<SYMBOL> IHL <ASPECT> มกรา <OPINION> เป๋าตุง <...
...,...,...
8308,GULFคาดรายได้ปีหน้าแตะ2แสนล้าน เหตุโรงไฟฟ้าทยอ...,<SYMBOL> GULF <ASPECT> รายได้ <OPINION> คาด <NEU>
8309,KEX ไตรมาส 3/66 ขาดทุน 889 ล้านบาท หวังปี 67 พ...,<SYMBOL> KEX <ASPECT> nan <OPINION> ขาดทุน <NEG>
8310,GPSC ไตรมาส 3/66 กำไรสุทธิ 1.7 พันล้าน โต 441%...,<SYMBOL> GPSC <ASPECT> กำไร <OPINION> โต <POS>
8311,TU ไตรมาส 3/66 กำไร 1.2 พันล้าน วูบ 52.3% - หั...,<SYMBOL> TU <ASPECT> กำไร <OPINION> วูบ <NEG>


In [ ]:
save_folder = '/content/drive/MyDrive/KMITL/Project/Data'
save_filename = file_dropdown.value
dataset.to_csv(os.path.join(save_folder, save_filename), index=False)